In [1]:
import math
import copy

A_FILE_PATH = "./data/a.txt"
B_FILE_PATH = "./data/b.txt"
C_FILE_PATH = "./data/c.txt"
D_FILE_PATH = "./data/d.txt"

SUM_FILE_PATH = "./data/aplusb.txt"
MULTIPLICATION_FILE_PATH = "./data/aorib.txt"
MULTIPLICATION2_FILE_PATH = "./data/corid.txt"

class RareMatrix:
    def __init__(self, from_matrix=None):
        
        if from_matrix:
            self.n = from_matrix.n
            self.rare_values = copy.deepcopy(from_matrix.rare_values)
        else:
            self.n = 0
            self.rare_values = {}
        
    def __setitem__(self, coords, value):
        if value != 0.0:
            if coords[0] in self.rare_values:
                self.rare_values[coords[0]][coords[1]] = value
            else:
                self.rare_values[coords[0]] = {coords[1]: value}
            
    def __getitem__(self, coords):
        if coords[0] in self.rare_values:
            if coords[1] in self.rare_values[coords[0]]:
                return self.rare_values[coords[0]][coords[1]]
        
        return 0.0
    
    def from_file_path(path):
        A = RareMatrix()
        
        with open(path, 'r') as f:
            A.n = int(f.readline())
            f.readline()
            
            for line in f:
                value, i, j = line.split(',')
                A[int(i), int(j)] = A[int(i), int(j)] + float(value)
        
        return A
    
    def sum_triag_matrix(self, B):
        A = RareMatrix(from_matrix=self)
        
        for i in range(0, B.n):
            A[i,i] = A[i,i] + B.a[i]
            
            if i < B.n - B.p:
                A[i, i + B.p] = A[i, i + B.p] + B.b[i]
            
            if i < B.n - B.q:
                A[i + B.q, i] = A[i + B.q, i] + B.c[i]
                
        return A
    
    def equals(self, B, error):
        for i in self.rare_values:
            for j in self.rare_values[i]:
                if math.fabs(self[i,j] - B[i,j]) > error:
                    return False
        
        return True
    
    def mul_triag_matrix(self, B):
        A = RareMatrix()
        
        for i in self.rare_values:
            for j in range(0, B.n):
                prod1 = B.a[j] * self[i,j]
                prod2 = 0.0
                prod3 = 0.0
                
                if j >= B.p:
                    prod2 = B.b[j - B.p] * self[i,j - B.p]
                
                if j < B.n - B.q:
                    prod3 = B.c[j] * self[i, j + B.q]
                    
                A[i,j] = prod1 + prod2 + prod3
        
        return A
    
class RareTriagMatrix:
    def __init__(self):
        self.n = 0
        self.p = 0
        self.q = 0
        self.a = []
        self.b = []
        self.c = []
    
    def from_file_path(path):
        A = RareTriagMatrix()
        
        with open(path, 'r') as f:
            A.n = int(f.readline())
            A.p = int(f.readline())
            A.q = int(f.readline())
            f.readline()
            
            for i in range(0, A.n):
                A.a.append(float(f.readline()))
            f.readline()
            
            for i in range(0, A.n - A.p):
                A.b.append(float(f.readline()))
            f.readline()
            
            for i in range(0, A.n - A.q):
                A.c.append(float(f.readline()))
            f.readline()
                
        return A
    
    def mul_triag_matrix(self, B):
        A = RareMatrix()
        
        for i in range(0, self.n):
            coords_A = [(i, self.a[i])]
            if i < self.n - self.p:
                coords_A.append((i + self.p, self.b[i]))
            if i >= self.q:
                coords_A.append((i - self.q, self.c[i - self.q]))
            
            for j in range(0, B.n):
                coords_B = [(j, B.a[j])]
                if j >= B.p:
                    coords_B.append((j - B.p, B.b[j - B.p]))
                if j < B.n - B.q:
                    coords_B.append((j + B.q, B.c[j]))
                    
                for coord_A in coords_A:
                    for coord_B in coords_B:
                        if coord_A[0] == coord_B[0]:
                            A[i,j] = A[i,j] + coord_A[1] * coord_B[1]
        
        return A


In [2]:
A = RareMatrix.from_file_path(A_FILE_PATH)
A.n, A.rare_values[0]

(2021, {0: 132.5, 1110: 7.0, 15: 17.0, 245: 8.0})

In [3]:
B = RareTriagMatrix.from_file_path(B_FILE_PATH)
len(B.a), len(B.b), len(B.c)

(2021, 2020, 2020)

In [4]:
my_A_plus_B = A.sum_triag_matrix(B)
my_A_plus_B.rare_values[1]

{626: 24.5, 878: 21.0, 1: 161.0, 187: 1.0, 937: 3.5, 0: 1.75, 2: 4.75}

In [5]:
read_A_plus_B = RareMatrix.from_file_path(SUM_FILE_PATH)
my_A_plus_B.equals(read_A_plus_B, 10 **(-10))

True

In [6]:
my_A_ori_B = A.mul_triag_matrix(B)
my_A_ori_B.rare_values[0]

{0: 1424.375,
 1: 397.5,
 14: 157.25,
 15: 89.25,
 16: 119.0,
 244: 68.0,
 245: 84.0,
 246: 48.0,
 1109: 8.75,
 1110: 14.0,
 1111: 89.25}

In [7]:
read_A_ori_B = RareMatrix.from_file_path(MULTIPLICATION_FILE_PATH)
read_A_ori_B.rare_values[0]

{0: 1424.375,
 14: 157.25,
 16: 119.0,
 244: 68.0,
 246: 48.0,
 1110: 14.0,
 1: 397.5,
 15: 89.25,
 245: 84.0,
 1109: 8.75,
 1111: 89.25}

In [8]:
my_A_ori_B.equals(read_A_ori_B, 10 **(-10))

True

In [9]:
C = RareTriagMatrix.from_file_path(C_FILE_PATH)
len(C.a), len(C.b), len(C.c)

(5, 3, 4)

In [10]:
D = RareTriagMatrix.from_file_path(D_FILE_PATH)
len(D.a), len(D.b), len(D.c)

(5, 3, 3)

In [11]:
my_C_ori_D = C.mul_triag_matrix(D)
my_C_ori_D.rare_values[0]

{0: 1164.0, 2: 585.0, 4: 83.5}

In [12]:
read_C_ori_D = RareMatrix.from_file_path(MULTIPLICATION2_FILE_PATH)
read_C_ori_D.rare_values[0]

{0: 1164.0, 2: 585.0, 4: 83.5}

In [13]:
my_C_ori_D.equals(read_C_ori_D, 10 **(-10))

True